In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

spark = SparkSession.builder.master('local[*]').appName('PySparkApplication').getOrCreate()
sc = spark.sparkContext

In [2]:
schema = 'letter: string, number: int'
data = [('a', 1), ('b', 2)]

rdd = sc.parallelize(data)
df = rdd.toDF(schema)
df = spark.createDataFrame(data, schema)

In [ ]:
path = 'C:/Users/bernard/Desktop/glassdoor.csv'
df = spark.read.csv(path, header=True, inferSchema=True)

df.printSchema()
df.select(F.max(df.Age)).show()
df = df.withColumns({
	'ReviewDate': F.to_date(df.ReviewDateKey, 'yyyyMMdd'),
	'ReviewDayOfTheWeek': F.date_format(F.to_date(df.ReviewDateKey, 'yyyyMMdd'), 'E'),
	'AgeGroup': F.when(df.Age < 20, 'teenager')
				.when(df.Age < 30, 'young adult')
				.when(df.Age < 40, 'adult')
				.otherwise('dead')
})

In [ ]:
df\
	.groupBy('AgeGroup', 'Gender', 'MaritalStatus')\
	.count()\
	.withColumnRenamed('count', 'NumberOfEmployees')\
	.orderBy('AgeGroup', 'Gender', 'MaritalStatus')\
	.show()

+-----------+------+-------------+-----------------+
|   AgeGroup|Gender|MaritalStatus|NumberOfEmployees|
+-----------+------+-------------+-----------------+
|      adult|Female|     Divorced|               16|
|      adult|Female|      Married|               30|
|      adult|Female|       Single|               38|
|      adult|  Male|     Divorced|               31|
|      adult|  Male|      Married|               57|
|      adult|  Male|       Single|               41|
|       dead|Female|     Divorced|               15|
|       dead|Female|      Married|               24|
|       dead|Female|       Single|               24|
|       dead|  Male|     Divorced|               23|
|       dead|  Male|      Married|               42|
|       dead|  Male|       Single|               28|
|   teenager|Female|       Single|                5|
|   teenager|  Male|       Single|                7|
|young adult|Female|     Divorced|                8|
|young adult|Female|      Married|            

In [18]:
df\
	.groupBy('Gender', 'AgeGroup', 'YearsAtCompany')\
	.agg(F.mean('MonthlyIncome').alias('AverageMonthlyIncome'), F.min('MonthlyIncome').alias('MinimumMonthlyIncome'))\
	.orderBy('Gender', 'AgeGroup', 'YearsAtCompany')\
	.show()

+------+--------+--------------+--------------------+--------------------+
|Gender|AgeGroup|YearsAtCompany|AverageMonthlyIncome|MinimumMonthlyIncome|
+------+--------+--------------+--------------------+--------------------+
|Female|   adult|             1|              3149.0|                1261|
|Female|   adult|             2|              2958.5|                2014|
|Female|   adult|             3|              3860.4|                2232|
|Female|   adult|             4|             3676.25|                2231|
|Female|   adult|             5|   5546.285714285715|                2132|
|Female|   adult|             6|              5560.5|                4425|
|Female|   adult|             7|              7045.0|                4084|
|Female|   adult|             8|  4079.8333333333335|                2153|
|Female|   adult|             9|             6779.25|                2707|
|Female|   adult|            10|   7161.454545454545|                2022|
|Female|   adult|        

In [22]:
df\
	.groupBy('EducationField', 'Department')\
	.count()\
	.withColumnRenamed('count', 'NumberOfEmployees')\
	.orderBy('EducationField', 'Department')\
	.show()

+----------------+--------------------+-----------------+
|  EducationField|          Department|NumberOfEmployees|
+----------------+--------------------+-----------------+
| Human Resources|     Human Resources|                7|
|   Life Sciences|     Human Resources|                2|
|   Life Sciences|Research & Develo...|              151|
|   Life Sciences|               Sales|               49|
|       Marketing|               Sales|               58|
|         Medical|     Human Resources|                3|
|         Medical|Research & Develo...|              129|
|         Medical|               Sales|               27|
|           Other|Research & Develo...|               18|
|           Other|               Sales|                5|
|Technical Degree|     Human Resources|                3|
|Technical Degree|Research & Develo...|               34|
|Technical Degree|               Sales|               14|
+----------------+--------------------+-----------------+



In [ ]:
df\
	.groupBy('ReviewDayOfTheWeek')\
	.count()\
	.withColumnRenamed('count', 'NumberOfReviews')\
	.orderBy('NumberOfReviews', ascending=False)\
	.show()

In [ ]:
spark.stop()